# Transfer Learning
Using VGG16

In [1]:
import tensorflow as tf
import keras
print('tensorflow version: ', tf.__version__)
print('keras version: ', keras.__version__)

tensorflow version:  2.9.2
keras version:  2.9.0


In [2]:
# define sess to use gpu
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Quadro P4000, pci bus id: 0000:00:05.0, compute capability: 6.1



In [3]:
# import necessary libraries and functions
import os
import joblib
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import Dense, InputLayer
from keras.models import Sequential

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
import sys

sys.path.append('..')
from utils.s3_class import S3Functions
s3_funcs = S3Functions(bucket_name='jdgallegoq-pinacle')

In [5]:
# create random number generator
seed = 42
rng = np.random.RandomState(seed)

In [6]:
# train data
train_data = pd.read_csv(s3_funcs.read_object(key='gender_clf/train/train.csv'))
print(train_data.info())

# test data
test_data = pd.read_csv(s3_funcs.read_object(key='gender_clf/test.csv'))
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12196 entries, 0 to 12195
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_names  12196 non-null  object
 1   class        12196 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 190.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5227 entries, 0 to 5226
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_names  5227 non-null   object
dtypes: object(1)
memory usage: 41.0+ KB
None


In [7]:
train_data['class'].value_counts()

1    6174
0    6022
Name: class, dtype: int64

In [8]:
# load images
IMAGES_PATH = 'gender_clf/train/images/'

X = []
not_found = []
num_images_read = 5000
#for img_name in tqdm(train_data.image_names):
for img_name in tqdm(train_data.image_names[:num_images_read]):
  img = s3_funcs.read_image(key=IMAGES_PATH+img_name)
  X.append(img)

X = np.array(X)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [9]:
# Model architecture
from keras.models import Model
from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.layers import (
    InputLayer,
    Dense
    )
from keras.models import Sequential
from keras.optimizers import Adam

In [10]:
# preprocess according to pretained model parameters
X = preprocess_input(X)
print(X.min(), X.max())
# define target as categorical (VGG16 target type)
y = train_data['class'].values[:num_images_read]
y = to_categorical(y)

In [11]:
# split dataset

X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=seed
)

del X

In [12]:
# Load architecture
# VGG16 was pretrained on MNIST and ImageNet
# so load ImageNet weights cause our clf problem
# is more suitable with it.
base_model = VGG16(weights='imagenet')

# show summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
# we're going to create a new model cause
# we need only 2 (only 2 categories) output not 1000.
# so, let's use all weights insted of last layers
x = Dense(100, activation='relu', name='my_dense')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='my_pred')(x)
my_model = Model(inputs=base_model.input, outputs=y)
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
# compile the model
adam = Adam(learning_rate=1e-5, clipvalue=1)
my_model.compile(
    loss='categorical_crossentropy',
    optimizer=adam,
    metrics=['accuracy'],
)

In [16]:
# train
epochs = 10
my_model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_valid, y_valid)
)

2024-04-11 23:48:07.095538: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2408448000 exceeds 10% of free system memory.
2024-04-11 23:48:09.306069: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2408448000 exceeds 10% of free system memory.


Epoch 1/10


2024-04-11 23:48:13.409826: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:48:13.409916: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:48:13.836795: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-11 23:48:14.535915: W tensorflow/core/common_runtime/bfc_allocato

125/125 [==============================] - 70s 513ms/step - loss: 0.8811 - accuracy: 0.6550 - val_loss: 0.5679 - val_accuracy: 0.7260
Epoch 2/10
125/125 [==============================] - 67s 533ms/step - loss: 0.3648 - accuracy: 0.8388 - val_loss: 0.4749 - val_accuracy: 0.7940
Epoch 3/10
125/125 [==============================] - 67s 536ms/step - loss: 0.2048 - accuracy: 0.9190 - val_loss: 0.4242 - val_accuracy: 0.8330
Epoch 4/10
125/125 [==============================] - 67s 537ms/step - loss: 0.1090 - accuracy: 0.9632 - val_loss: 0.4077 - val_accuracy: 0.8440
Epoch 5/10
125/125 [==============================] - 67s 537ms/step - loss: 0.0583 - accuracy: 0.9868 - val_loss: 0.4648 - val_accuracy: 0.8340
Epoch 6/10
125/125 [==============================] - 67s 537ms/step - loss: 0.0266 - accuracy: 0.9967 - val_loss: 0.4480 - val_accuracy: 0.8600
Epoch 7/10
125/125 [==============================] - 67s 537ms/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.4673 - val_accuracy: 0.8

In [22]:
# validate metrics
preds = my_model.predict(X_valid)
predicted_class = np.argmax(preds, axis=1)
accuracy_score(y_valid[:,1], predicted_class)

32/32 [==============================] - 5s 166ms/step


0.862